# Setup

Fall [these instructions](https://docs.astral.sh/uv/getting-started/installation/) to install `uv`. Then run the following command to set up your environment:

```bash
uv sync --quiet
```

In [1]:
# Add the parent directory to the system path
import sys
from pathlib import Path
sys.path.append(str(Path("../../").resolve()))
# Imports
from robora import Workflow, SQLiteStorageProvider
from robora.sonar_query import SonarQueryHandler
from robora.mock_query import MockQueryHandler
from library.organization_question import OrganizationModel, get_question_set
from data import COUNTRIES, DOMAINS
import pandas as pd


In [2]:
sqlstorage = SQLiteStorageProvider("organization.db")
workflow = Workflow(
    SonarQueryHandler(OrganizationModel),
    sqlstorage,
    workers = 4,
    )
question_set = get_question_set(
    domains=['Justice'],
    countries=COUNTRIES,
)
question_set.max_questions = 0  # No limit

Loaded 193 stored question hashes from organization.db


In [3]:
await workflow.ask_multiple(question_set, return_results=False)

ask_multiple: gathering answers
ask_multiple_stream: starting for 193 questions with 4 workers
Processing question 1/193: What is the top-level state Organ (i.e., ministry/department/agency) responsible for {domain} in {country}? - {'domain': 'JUSTICE', 'country': 'BENIN'}
Processing question 2/193: What is the top-level state Organ (i.e., ministry/department/agency) responsible for {domain} in {country}? - {'domain': 'JUSTICE', 'country': 'GUATEMALA'}
Processing question 3/193: What is the top-level state Organ (i.e., ministry/department/agency) responsible for {domain} in {country}? - {'domain': 'JUSTICE', 'country': 'NAURU'}
Processing question 4/193: What is the top-level state Organ (i.e., ministry/department/agency) responsible for {domain} in {country}? - {'domain': 'JUSTICE', 'country': 'THAILAND'}
Found cached response
Using cached response
QueryResponse(full_response={'id': '675665b8-4442-4452-937c-8c199e9fdcae', 'model': 'sonar', 'created': 1761182160, 'usage': {'prompt_toke

[]

In [4]:
answers = []
async for answer in workflow.dump_answers():
    answers.append(answer)
flattened = pd.concat([ans.flattened for ans in answers]).reset_index(drop=True)
flattened.drop(columns=["enriched_citations"], inplace=True)
flattened

,question,error,domain,country,organization_name,confidence,pretty_citations
0,"What is the top-level state Organ (i.e., minis...",None,JUSTICE,ALBANIA,Ministry of Justice / Ministria e Drejtësisë,ConfidenceLevel.HIGH,[1] ️Ministry of Justice / Ministria e Drejtës...
1,"What is the top-level state Organ (i.e., minis...",None,JUSTICE,AFGHANISTAN,Ministry of Justice of the Islamic Emirate of ...,ConfidenceLevel.HIGH,[1] Ministry of Justice (Afghanistan) - Devex ...
2,"What is the top-level state Organ (i.e., minis...",None,JUSTICE,TANZANIA,Ministry of Justice and Constitutional Affairs,ConfidenceLevel.HIGH,[1] Tanzanian Minister of Justice visits the M...
3,"What is the top-level state Organ (i.e., minis...",None,JUSTICE,LEBANON,Ministry of Justice,ConfidenceLevel.HIGH,[1] Ministry of Justice (Lebanon) - Devex (htt...
4,"What is the top-level state Organ (i.e., minis...",None,JUSTICE,SAINT VINCENT AND THE GRENADINES,Ministry of Legal Affairs,ConfidenceLevel.HIGH,[1] Judicial System in St Vincent and the Gren...
...,...,...,...,...,...,...,...
188,"What is the top-level state Organ (i.e., minis...",None,JUSTICE,MYANMAR,Ministry of Legal Affairs,ConfidenceLevel.HIGH,[1] Ministry of Legal Affairs (Myanmar) - Wiki...
189,"What is the top-level state Organ (i.e., minis...",None,JUSTICE,CUBA,Ministry of Justice of Cuba,ConfidenceLevel.HIGH,[1] Ministry of Justice (Cuba) - Wikipedia (ht...
190,"What is the top-level state Organ (i.e., minis...",None,JUSTICE,LATVIA,Ministry of Justice of the Republic of Latvia,ConfidenceLevel.HIGH,[1] Ministry of Justice (Latvia) - Wikipedia (...
191,"What is the top-level state Organ (i.e., minis...",None,JUSTICE,GRENADA,"Ministry of Legal Affairs, Labour and Consumer...",ConfidenceLevel.HIGH,[1] Judicial System in Grenada - Commonwealth ...


In [5]:
wrangled = flattened.copy()


In [6]:
wrangled.to_csv("organization_names_justice.csv", index=False)